In [2]:
import numpy as np
import sys
from ipycanvas import Canvas
from tqdm import tqdm
sys.path.append('../src')
from util import draw, grab_plot, plot, RunTime, VideoWriter

In [24]:
CANVAS_SIZE = 500

def update():
    global farm, canvas, x, v, a

    x, v = farm(x, v, a)
    draw(x, canvas, CANVAS_SIZE, ant_size=5)

def toggle(*_):
    global runtime 

    if runtime.running:
        runtime.stop()
    else:
        runtime.start()

def simulate(function, interval=.02):
    global runtime, canvas, x

    display(canvas)
    draw(x, canvas, CANVAS_SIZE, ant_size=5)

    try:
        runtime.stop()
    except:
        pass

    runtime = RunTime(function, interval)

def record(n_steps=500):
    global farm, x, v

    with VideoWriter(size=CANVAS_SIZE) as vid:
        for _ in tqdm(range(n_steps), desc='Ant Farm'):
            plot(x)
            vid.write(grab_plot())
            x, v = farm(x, v)

In [59]:
class SandboxFarm:
    def __init__(self, dt=.1, figsize=500):
        self.dt = dt
        self.figsize = figsize

    def __call__(self, x, v, a):
        in_wall = False

        for i in range(x.shape[0]):
            if x[i, 0] > self.figsize - 5.:
                v[i, 0] = -.9*np.abs(v[i, 0])
                v[i, 1] = .99*v[i, 1]
                in_wall = True
            elif x[i, 0] < 5.:
                v[i, 0] = .9*np.abs(v[i, 0])
                v[i, 1] = .99*v[i, 1]
                in_wall = True
            else:
                v[i, 0] = v[i, 0] + a[i, 0]*self.dt
                
            if x[i, 1] > self.figsize - 5.:
                v[i, 0] = .99*v[i, 0]
                v[i, 1] = -.9*np.abs(v[i, 1])
                in_wall = True
            elif x[i, 1] < 5.:
                v[i, 0] = .99*v[i, 0]
                v[i, 1] = .9*np.abs(v[i, 1])
                in_wall = True
            else:
                v[i, 1] = v[i, 1] + a[i, 1]*self.dt

        if in_wall:
            x = x + v*self.dt
        else:
            x = x + v*self.dt + .5*self.dt**2

        return x, v

In [65]:
N_ANTS = 10

farm = SandboxFarm(figsize=CANVAS_SIZE)
x = .5*CANVAS_SIZE*np.ones((N_ANTS, 2))
# v = np.zeros_like(x)
v = -100.*np.random.random(x.shape) + 50.
a = np.array([[0., 9.8],]*N_ANTS)
canvas = Canvas(width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas.on_mouse_down(toggle)

simulate(update)

Canvas(width=500)